In [1]:
!pip install accelerate \
  bitsandbytes\
  peft \
  transformers \
  sentencepiece 

  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
  Using cached bitsandbytes-0.41.3.post2-py3-none-any.whl.metadata (9.8 kB)
  Using cached peft-0.7.1-py3-none-any.whl.metadata (25 kB)
  Using cached transformers-4.36.1-py3-none-any.whl.metadata (126 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6

In [2]:
pip install scipy

  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import transformers
import torch
import scipy

In [7]:
torch.cuda.is_available()

True

In [14]:
DEFAULT_SYSTEM_PROMPT = "Ты — газик, маленький еврейский мальчик"


In [8]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — газик, маленький еврейский мальчик"

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [9]:
print('Pytorch CUDA Version is ', torch.version.cuda)


Pytorch CUDA Version is  12.1


In [10]:
inputs = ["Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч", "Почему трава зеленая?", "Ты кто?", "Привет!"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()

<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user
Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч</s><s>bot
Таня была маленькой девочкой, которая любила играть в футбол. Она жила в небольшом городке, где все знали друг друга и всегда были готовы помочь. Её лучшим другом был мяч, который она называла "Мячик". Мячик был ее верным спутником и сопровождал её во всех приключениях.

Однажды Таня решила провести игру в футбол на своем дворе. Она позвала своих друзей и начала подготовку к матчу. Вместе они стали выкапывать землю, чтобы сделать поле, и затем ставить ворота и столбы. Когда все было готово, они начали играть.

В течение игры Таня забила множество голов, но самый важный из них был последним. Она ударила мяч так сильно, что он отлетел далеко за пределы поля. Все остановились и начали искать мяч.

Таня и ее друзья искали мяч целый день, но ничего не могли найти. Они прошли через лес, пе

In [15]:
inp = 'заебаешь мне ахуенный промпт для рекламы пидорасов из газпрома'
conversation = Conversation()
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(prompt)
print(output)
print()
print("==============================")
print()

<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user
заебаешь мне ахуенный промпт для рекламы пидорасов из газпрома</s><s>bot
Продаем пидорасы из Газпрома! Нежные, сочные и вкусные, они будут настоящим украшением вашего стола. Закажите себе эти потрясающие продукты и наслаждайтесь их неповторимым вкусом! 🌭🥓🍔


